In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE116833"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE116833"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE116833.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE116833.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE116833.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral gene expression in cocaine use disorders with low and high anhedonia"
!Series_summary	"Treatments for Cocaine Use Disorder (CUD) are variably effective, and there are no FDA-approved medications. One approach to developing new treatments for CUD may be to investigate and target poor prognostic signs. One such sign is anhedonia (i.e. a loss of pleasure or interest in non-drug rewards), which predicts worse outcomes in existing CUD treatments. Inflammation is thought to underlie anhedonia in many other disorders, but the relationship between anhedonia and inflammation has not been investigated in CUD. Therefore, we assessed peripheral genome-wide gene expression in n = 48 individuals with CUD with high (n=24) vs. low (n = 24) levels of anhedonia, defined by a median split of self-reported anhedonia. Our hypothesis was that individuals with high anhedonia would show differential gene expression in inflammatory pathways. No individual genes

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os

# 1. Gene Expression Data Availability
# Based on the background information, this dataset uses "HumanHT-12 v4.0 Gene Expression BeadChip" 
# which indicates it contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Substance Use Disorder), we can use the anhedonia level as a proxy
# since this study is about Cocaine Use Disorder with high vs low anhedonia
trait_row = 0  # anhedonia level (high/low)
age_row = 2    # age data is available
gender_row = 1 # gender data is available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert anhedonia level to binary representation."""
    if pd.isna(value):
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
    if value.lower() == "low":
        return 0
    elif value.lower() == "high":
        return 1
    return None

def convert_age(value):
    """Convert age string to numeric value."""
    if pd.isna(value):
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary representation (female=0, male=1)."""
    if pd.isna(value):
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Extract clinical features from the clinical_data that was already loaded
        # in a previous step and is available in the environment
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Save the extracted clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")


Preview of extracted clinical features:
{'GSM3262437': [0.0, 40.0, 0.0], 'GSM3262438': [0.0, 50.0, 1.0], 'GSM3262439': [0.0, 46.0, 1.0], 'GSM3262440': [0.0, 38.0, 1.0], 'GSM3262441': [0.0, 52.0, 1.0], 'GSM3262442': [0.0, 51.0, 1.0], 'GSM3262443': [0.0, 33.0, 0.0], 'GSM3262444': [0.0, 49.0, 1.0], 'GSM3262445': [1.0, 52.0, 1.0], 'GSM3262446': [1.0, 52.0, 1.0], 'GSM3262447': [0.0, 44.0, 1.0], 'GSM3262448': [0.0, 49.0, 1.0], 'GSM3262449': [1.0, 51.0, 0.0], 'GSM3262450': [0.0, 40.0, 0.0], 'GSM3262451': [0.0, 34.0, 0.0], 'GSM3262452': [0.0, 53.0, 1.0], 'GSM3262453': [1.0, 45.0, 1.0], 'GSM3262454': [1.0, 56.0, 1.0], 'GSM3262455': [0.0, 36.0, 0.0], 'GSM3262456': [0.0, 52.0, 1.0], 'GSM3262457': [1.0, 57.0, 1.0], 'GSM3262458': [0.0, 48.0, 1.0], 'GSM3262459': [0.0, 52.0, 1.0], 'GSM3262460': [0.0, 30.0, 1.0], 'GSM3262461': [0.0, 28.0, 1.0], 'GSM3262462': [1.0, 53.0, 1.0], 'GSM3262463': [1.0, 46.0, 1.0], 'GSM3262464': [1.0, 35.0, 0.0], 'GSM3262465': [0.0, 55.0, 1.0], 'GSM3262466': [0.0, 57.0, 1.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 72
Header line: "ID_REF"	"GSM3262437"	"GSM3262438"	"GSM3262439"	"GSM3262440"	"GSM3262441"	"GSM3262442"	"GSM3262443"	"GSM3262444"	"GSM3262445"	"GSM3262446"	"GSM3262447"	"GSM3262448"	"GSM3262449"	"GSM3262450"	"GSM3262451"	"GSM3262452"	"GSM3262453"	"GSM3262454"	"GSM3262455"	"GSM3262456"	"GSM3262457"	"GSM3262458"	"GSM3262459"	"GSM3262460"	"GSM3262461"	"GSM3262462"	"GSM3262463"	"GSM3262464"	"GSM3262465"	"GSM3262466"	"GSM3262467"	"GSM3262468"	"GSM3262469"	"GSM3262470"	"GSM3262471"	"GSM3262472"	"GSM3262473"	"GSM3262474"	"GSM3262475"	"GSM3262476"	"GSM3262477"	"GSM3262478"	"GSM3262479"	"GSM3262480"	"GSM3262481"	"GSM3262482"	"GSM3262483"	"GSM3262484"
First data line: "ILMN_1343291"	24685.7	24733.9	28152.7	24733.9	29971.7	24595.2	20213.5	24473.2	23915.6	23099	28882.2	29971.7	29239.2	28882.2	25456.1	24922.5	23915.6	29239.2	27115	19761.6	27637.4	28552.7	27988	27492.2	25378.4	22610.7	25623.6	27355.8	24272.2	26606.5	27816.5	27988	23723.4	20329.9	21227.4	22996.8	24874.5	24021

### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, these are Illumina probe IDs (starting with "ILMN_") 
# rather than standard human gene symbols.
# These identifiers need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE116833
Line 6: !Series_title = Peripheral gene expression in cocaine use disorders with low and high anhedonia
Line 7: !Series_geo_accession = GSE116833
Line 8: !Series_status = Public on Nov 02 2018
Line 9: !Series_submission_date = Jul 09 2018
Line 10: !Series_last_update_date = Nov 30 2018
Line 11: !Series_pubmed_id = 30408130
Line 12: !Series_summary = Treatments for Cocaine Use Disorder (CUD) are variably effective, and there are no FDA-approved medications. One approach to developing new treatments for CUD may be to investigate and target poor prognostic signs. One such sign is anhedonia (i.e. a loss of pleasure or interest in non-drug rewards), which predicts worse outcomes in existing CUD treatme


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# From inspecting the gene annotation preview, we need to use ID for probe identifiers
# and Symbol for gene symbols

# 1. Define the columns for probe IDs and gene symbols in the gene annotation dataframe
id_column = 'ID'
symbol_column = 'Symbol'

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, id_column, symbol_column)
print("\nGene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene expression data preview (first 5 genes):")
print(gene_data.head())

# Check the number of genes after mapping
print(f"\nNumber of genes after mapping: {len(gene_data)}")

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")



Gene mapping preview (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Gene expression data preview (first 5 genes):
       GSM3262437  GSM3262438  GSM3262439  GSM3262440  GSM3262441  GSM3262442  \
Gene                                                                            
A1BG        238.2       263.3       264.4       247.0       262.2       274.6   
A1CF        354.6       344.1       354.7       363.3       381.6       364.4   
A26C3       365.0       330.3       354.6       340.1       356.3       328.8   
A2BP1       455.3       452.7       452.7       450.8       468.0       438.1   
A2LD1       208.9       227.4       242.2       214.2       239.1       229.1   

       GSM3262443  GSM3262444  GSM3262445  GSM3262446  ...  GSM3262475  \
Gene                       

Gene expression data saved to: ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE116833.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (21464, 48)
Gene data shape after normalization: (20259, 48)


Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE116833.csv
Raw clinical data shape: (5, 49)
Clinical features:
                        GSM3262437  GSM3262438  GSM3262439  GSM3262440  \
Substance_Use_Disorder         0.0         0.0         0.0         0.0   
Age                           40.0        50.0        46.0        38.0   
Gender                         0.0         1.0         1.0         1.0   

                        GSM3262441  GSM3262442  GSM3262443  GSM3262444  \
Substance_Use_Disorder         0.0         0.0         0.0         0.0   
Age                           52.0        51.0        33.0        49.0   
Gender                         1.0         1.0         0.0         1.0   

                        GSM3262445  GSM3262446  ...  GSM3262475  GSM3262476  \
Substance_Use_Disorder         1.0         1.0  ...         1.0         0.0   
Age                           52.0        52.0  ...        41.0        31.0   
Gender          

Data shape after handling missing values: (48, 20262)
For the feature 'Substance_Use_Disorder', the least common label is '0.0' with 24 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.75
  50% (Median): 48.0
  75%: 52.0
Min: 24.0
Max: 59.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 11 occurrences. This represents 22.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Substance_Use_Disorder/cohort_info.json


Linked data saved to ../../output/preprocess/Substance_Use_Disorder/GSE116833.csv
